In [1]:
import time
import requests
import pandas as pd

In [2]:
# constants 
collection_orderbook = 'BENrqx18n8tP2xYPBQ7vaDz88Y7wSmzB1xkr8h4XrhvM'  
total_transactions = 100 
limit_balances = 100
url = "https://api.mainnet-beta.solana.com"
headers = {"Content-Type": "application/json"}
quick_break = 15

In [3]:
# transactions of orderbook
payload = {"jsonrpc": "2.0", "id": 1, "method": "getSignaturesForAddress", "params": [collection_orderbook, {"limit": total_transactions }]}
r = requests.post(url, json=payload, headers=headers)
data = r.json()
signature_list = []
for x in data['result']:
    if x['confirmationStatus'] != 'processed':
        signature_list.append(x['signature'])
time.sleep(quick_break)
if r.status_code != 200: print(r.status_code) 

In [4]:
# offered loans 
blocktime_list = []
user_list = []
escrow_list = []
for i in signature_list:
    payload = {"jsonrpc": "2.0", "id": 1, "method": "getTransaction", "params": [i, "json"]}
    r = requests.post(url, json=payload, headers=headers)
    data = r.json()
    if list(data.keys())[1] == 'result':
        count = 0
        for x in data['result']['meta']['logMessages']:
            if 'OfferLoan' in x:
                count += 1 
        blocktime = data['result']['blockTime'] 
        accounts = data['result']['transaction']['message']['accountKeys'] 
        instructions = data['result']['transaction']['message']['instructions']
        if count == 1:
            blocktime_list.append(blocktime)
            user_list.append(accounts[0]) 
            escrow_list.append(accounts[instructions[2]['accounts'][4]])   
        if count == 3:
            blocktime_list.extend([blocktime, blocktime, blocktime])
            user_list.extend([accounts[0], accounts[0], accounts[0]]) 
            escrow_list.extend([accounts[instructions[2]['accounts'][4]], accounts[instructions[3]['accounts'][4]], accounts[instructions[4]['accounts'][4]]])      
        if count == 9:
            print('9 loans')
    if r.status_code != 200: 
        print(r.status_code)
        break   
    time.sleep(quick_break)

In [5]:
# offered amount of loans 
amount_list = []
for x in range(0, len(escrow_list), limit_balances):
    chunk = escrow_list[x:x+limit_balances]
    payload = {"jsonrpc": "2.0", "id": 1, "method": "getMultipleAccounts", "params": [chunk,{"encoding": "base58"}]}
    r = requests.post(url, json=payload, headers=headers)
    data = r.json()
    for x in data['result']['value']:
        if x is not None:
            amount_list.append(x['lamports'])
        else:
            amount_list.append(0)  
    if r.status_code != 200: 
        print(r.status_code)
        break           
    time.sleep(quick_break)

In [6]:
# top loan offers
offers = pd.DataFrame({'blocktime': blocktime_list, 'amount': amount_list, 'user': user_list, 'escrow': escrow_list,})
offers['amount'] *= 0.000000001
offers['amount'] = round(offers['amount'], 2)
offers['user_readable'] = offers['user'].str[0:3] + '___' + offers['user'].str[-3:]
offers['datetime'] = pd.to_datetime(offers['blocktime'], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
offers_desc = offers[offers['amount'] > 0.01][['amount', 'user_readable', 'datetime']].sort_values(['amount', 'datetime'], ascending=[False, True]).reset_index(drop=True)
offers_desc.index += 1
offers_desc

,amount,user_readable,datetime
1,0.43,H9k___P3M,2024-10-29 05:10:33
2,0.42,GYe___Jzn,2024-10-29 01:04:56
3,0.42,GYe___Jzn,2024-10-29 01:04:56
4,0.42,GYe___Jzn,2024-10-29 01:04:56
5,0.37,DGL___tgA,2024-10-29 12:36:13
6,0.37,DGL___tgA,2024-10-29 14:10:53
7,0.36,2n6___dgB,2024-10-25 10:09:40
8,0.36,2n6___dgB,2024-10-25 10:09:40
9,0.26,DGL___tgA,2024-10-29 14:12:03


In [7]:
# last updated 2024/10/29
# this notebook uses a public rpc api, if in the cel output below you are getting a 429, you have been rate limited, in that case please work with a private rpc api and replace the url variable in the second cel 
r.status_code

200